In [1]:
import numpy as np
import cupy as cp
print(cp.__version__)
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys
from ltedt import local_thickness

13.3.0


In [2]:
binary_volume = tiff.imread("/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume.tif")
# binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume.tif")
# Get the size of the volume
print(f'Size of the volume is: {binary_volume.shape}')

Size of the volume is: (1035, 1706, 1510)


In [3]:
# Cut the volume in a smaller part
binary_volume_bottom_part = binary_volume[300:500,600:800,200:400]

# output_path = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume_smaller.tif'
output_path = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller.tif'
tiff.imwrite(output_path, binary_volume_bottom_part)

In [4]:
smaller_skeleton = skeletonize(binary_volume_bottom_part)

# binarize the skeleton
smaller_skeleton = np.where(smaller_skeleton == True, 1, 0)

smaller_skeleton = smaller_skeleton.astype(np.uint8)


# output_path = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume_smaller_skeleton.tif'
output_path = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller_skeleton.tif'
tiff.imwrite(output_path, smaller_skeleton)

In [5]:
# Implement thickness with cuda
thickness = local_thickness(binary_volume_bottom_part, implementation="cupy")


In [6]:
# Skeleton mask
thickness_map = thickness*smaller_skeleton


# Calcular el valor máximo en thickness_map, excluyendo los ceros
max_distance = np.max(thickness_map[thickness_map > 0])

print(f"Max distance in thickness map: {max_distance}")

Max distance in thickness map: 6


In [7]:
principal_skel = Skeleton(smaller_skeleton)
principal_branch_data = summarize(principal_skel)

branch_type_0 = principal_branch_data[principal_branch_data['branch-type'] == 0]
branch_type_1 = principal_branch_data[principal_branch_data['branch-type'] == 1]
branch_type_2 = principal_branch_data[principal_branch_data['branch-type'] == 2]
branch_type_3 = principal_branch_data[principal_branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


/tmp/ipykernel_1010945/3061349273.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  principal_branch_data = summarize(principal_skel)


Number of branch type 0: 0
Number of branch type 1: 67
Number of branch type 2: 64
Number of branch type 3: 0


## Branch 1 type delete 40 pixels length

In [8]:
import numpy as np

def neighbors(coord):
    x, y, z = coord
    return [
        # 6 primary (axis-aligned) neighbors
        (x.item() + 1, y.item(), z.item()), (x.item() - 1, y.item(), z.item()), 
        (x.item(), y.item() + 1, z.item()), (x.item(), y.item() - 1, z.item()),
        (x.item(), y.item(), z.item() + 1), (x.item(), y.item(), z.item() - 1),
        
        # 12 diagonal neighbors (changing two axes)
        (x.item() + 1, y.item() + 1, z.item()), (x.item() - 1, y.item() - 1, z.item()),
        (x.item() + 1, y.item(), z.item() + 1), (x.item() - 1, y.item(), z.item() - 1),
        (x.item(), y.item() + 1, z.item() + 1), (x.item(), y.item() - 1, z.item() - 1),
        (x.item() + 1, y.item() - 1, z.item()), (x.item() - 1, y.item() + 1, z.item()),
        (x.item() + 1, y.item(), z.item() - 1), (x.item() - 1, y.item(), z.item() + 1),
        (x.item(), y.item() + 1, z.item() - 1), (x.item(), y.item() - 1, z.item() + 1)
    ]

# Define the is_junction_point function to identify junction points
def is_junction_point(coord, skeleton):
    neighboring_coords = neighbors(coord)
    counter = 0
    for neigh in neighboring_coords:
        if skeleton[neigh[0], neigh[1], neigh[2]].item() > 0:
            counter = counter + 1
    
    if counter > 2:
        return True
    else:
        return False

    # skeleton_neighbors = sum([skeleton[tuple(neigh)] > 0 for neigh in neighboring_coords])
    # return skeleton_neighbors.item() > 2  # A junction point has more than 2 neighbors

# Define the main function to remove branches of type 1 based on the criteria
def remove_branches(skeleton, branch_data, skel, thickness_map, length_threshold, thickness_threshold):
    """
    Removes branches of type 1 from the skeleton based on length and thickness criteria,
    while preserving junction points.
    
    Parameters:
    skeleton (np.array): The 3D skeleton array to clean.
    branch_data (pd.DataFrame): DataFrame with branch information including branch type and distance.
    skel (Skeleton): Skeleton object to access branch coordinates.
    thickness_map (np.array): Array containing thickness values for each voxel.
    length_threshold (float): Maximum length for branches to consider for deletion.
    thickness_threshold (float): Minimum thickness for branches to consider for deletion.
    
    Returns:
    np.array: A cleaned version of the skeleton with specified branches removed.
    """
    # Copy the skeleton to avoid modifying the original
    skeleton_cleaned = np.copy(skeleton)
    
    # Iterate over each branch
    for branch_id in branch_data.index:
        # Only process branches of type 1
        if branch_data.loc[branch_id, 'branch-type'] == 1:
            branch_length = branch_data.loc[branch_id, 'branch-distance']
            
            # Check length condition
            if branch_length <= length_threshold:
                coordinates = skel.path_coordinates(branch_id)
                
                thickness_values = []
                for coord in coordinates:
                    thick_val = thickness_map[coord[0], coord[1], coord[2]]
                    thickness_values.append(thick_val.item())

                # Check thickness condition
                if min(thickness_values) <= thickness_threshold:
                    # Remove the branch, skipping junction points
                    for coord in coordinates:
                        # rounded_coord = tuple(np.round(coord).astype(int))
                        if not is_junction_point(coord, skeleton):
                            skeleton_cleaned[coord[0], coord[1], coord[2]] = 0
    
    return skeleton_cleaned


def print_branches(skeleton_cleaned):
    skel = Skeleton(skeleton_cleaned)
    branch_data = summarize(skel)

    branch_type_0 = branch_data[branch_data['branch-type'] == 0]
    branch_type_1 = branch_data[branch_data['branch-type'] == 1]
    branch_type_2 = branch_data[branch_data['branch-type'] == 2]
    branch_type_3 = branch_data[branch_data['branch-type'] == 3]

    print(f"Number of branch type 0: {len(branch_type_0)}")
    print(f"Number of branch type 1: {len(branch_type_1)}")
    print(f"Number of branch type 2: {len(branch_type_2)}")
    print(f"Number of branch type 3: {len(branch_type_3)}")


In [9]:
x = 40
thickness_threshold = 4

skeleleton_clean_x40_th4 = remove_branches(skeleton=smaller_skeleton, 
                                           branch_data=principal_branch_data, 
                                           skel=principal_skel, 
                                           thickness_map=thickness_map, 
                                           length_threshold=x, 
                                           thickness_threshold=thickness_threshold)
print_branches(skeleleton_clean_x40_th4)

output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/skeleleton_clean_x40_th4.tif'
tiff.imwrite(output_path_cleaned, skeleleton_clean_x40_th4)

Number of branch type 0: 6
Number of branch type 1: 31
Number of branch type 2: 29
Number of branch type 3: 2


/tmp/ipykernel_1010945/4125157522.py:84: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [10]:
x = 20
thickness_threshold = 5

skeleleton_clean_x20_th5 = remove_branches(skeleton=skeleleton_clean_x40_th4, 
                                           branch_data=principal_branch_data, 
                                           skel=principal_skel, 
                                           thickness_map=thickness_map, 
                                           length_threshold=x, 
                                           thickness_threshold=thickness_threshold)
print_branches(skeleleton_clean_x20_th5)

output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/skeleleton_clean_x20_th5.tif'
tiff.imwrite(output_path_cleaned, skeleleton_clean_x20_th5)

Number of branch type 0: 11
Number of branch type 1: 27
Number of branch type 2: 17
Number of branch type 3: 1


/tmp/ipykernel_1010945/4125157522.py:84: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [11]:
skel = Skeleton(skeleleton_clean_x20_th5)
branch_data = summarize(skel)

for branch_id in branch_data.index:
    if branch_data.loc[branch_id, 'branch-type'] == 0:
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        print(branch_length)

48.35873884767937
12.949382989376325
1.4142135623730951
3.82842712474619
1.7320508075688772
3.82842712474619
7.706742302257039
21.070338854006465
2.8284271247461903
11.217332181807448
2.8284271247461903


/tmp/ipykernel_1010945/2796071380.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [12]:
import cc3d
import numpy as np

labels_out = cc3d.connected_components(skeleleton_clean_x20_th5)
labels_out, N = cc3d.connected_components(skeleleton_clean_x20_th5, return_N=True)
print(N)

22


In [13]:
# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


In [14]:
def bresenham_line_3d(p1, p2):
    # Implementa el algoritmo de Bresenham en 3D para trazar una línea entre dos puntos
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    points = []

    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    sz = 1 if z1 < z2 else -1
    err = dx - dy - dz

    while True:
        points.append((x1, y1, z1))
        if (x1, y1, z1) == (x2, y2, z2):
            break
        e2 = err * 2
        if e2 > -dy - dz:
            err -= dy
            x1 += sx
        if e2 < dx - dz:
            err += dx
            y1 += sy
        if e2 < dx - dy:
            err += dz
            z1 += sz
    return points


In [15]:
import numpy as np
from scipy.spatial.distance import cdist

# Paso 1: Calcular centroides de los blobs en 3D
def calculate_centroid(blob_labels, labels_out):
    centroids = {}
    for blob in np.unique(blob_labels):
        # Encuentra las coordenadas de todos los píxeles de ese blob
        coords = np.array(np.where(labels_out == blob)).T
        # Calcula el centroide como el promedio de las coordenadas
        centroid = np.mean(coords, axis=0)
        centroids[blob] = centroid
    return centroids

# Paso 2: Encontrar los blobs más cercanos en 3D
def find_closest_blobs(centroids):
    blobs = list(centroids.keys())
    distances = cdist([centroids[blob] for blob in blobs], [centroids[blob] for blob in blobs])
    np.fill_diagonal(distances, np.inf)  # Evitar que los blobs se conecten consigo mismos
    closest_pairs = np.unravel_index(np.argmin(distances), distances.shape)
    return blobs[closest_pairs[0]], blobs[closest_pairs[1]]

# Paso 3: Conectar los blobs más cercanos con una línea recta 3D
def connect_blobs_3d(labels_out, pixel_blob_mapping):
    # Calcular centroides de todos los blobs
    centroids = calculate_centroid(labels_out, labels_out)

    # Encontrar el par de blobs más cercanos
    blob1, blob2 = find_closest_blobs(centroids)

    # Encuentra un punto en cada blob
    coords_blob1 = np.array(np.where(labels_out == blob1)).T
    coords_blob2 = np.array(np.where(labels_out == blob2)).T

    # Seleccionar el primer píxel de cada blob
    p1 = coords_blob1[0]
    p2 = coords_blob2[0]

    # Calcular la línea de Bresenham entre los puntos (en 3D)
    line_points = bresenham_line_3d(p1, p2)

    # Conectar los blobs en el mapeo de píxeles
    for point in line_points:
        pixel_blob_mapping[tuple(point)] = blob1  # O puedes usar una mezcla de ambos blobs


In [17]:
connect_blobs_3d(labels_out, pixel_blob_mapping)

## Codigo pasado!

In [ ]:
x = 40  # Número mínimo de píxeles de la rama
thickness_threshold = 4  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_2 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_2[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_2 = skeleton_cleaned_2.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_2.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_2)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [ ]:
skel = Skeleton(skeleton_cleaned_2)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


Number of branch type 0: 6
Number of branch type 1: 59
Number of branch type 2: 50
Number of branch type 3: 0


/tmp/ipykernel_2882629/1015276533.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [ ]:
x = 60  # Número mínimo de píxeles de la rama
thickness_threshold = 6  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_3 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_3[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_3 = skeleton_cleaned_3.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_3.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_3)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [ ]:
skel = Skeleton(skeleton_cleaned_3)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


Number of branch type 0: 16
Number of branch type 1: 29
Number of branch type 2: 17
Number of branch type 3: 0


/tmp/ipykernel_2882629/1867985952.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [ ]:
x = 40  # Número mínimo de píxeles de la rama
thickness_threshold = 6  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_4 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_4[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_4 = skeleton_cleaned_4.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_4.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_4)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [ ]:
skel = Skeleton(skeleton_cleaned_4)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")

Number of branch type 0: 11
Number of branch type 1: 52
Number of branch type 2: 34
Number of branch type 3: 0


/tmp/ipykernel_2882629/3158189669.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [ ]:
x = 40  # Número mínimo de píxeles de la rama
thickness_threshold = 5  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_5 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_5[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_5 = skeleton_cleaned_5.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_5.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_5)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [ ]:
skel = Skeleton(skeleton_cleaned_5)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")

Number of branch type 0: 17
Number of branch type 1: 36
Number of branch type 2: 21
Number of branch type 3: 0


/tmp/ipykernel_2882629/2138058238.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)
